# 图像增广（Data augmentation）
图像增广一般用来人工产生不同的图像，比如对图像进行旋转、翻转、随机裁剪、缩放等等。这里我们选择在训练阶段对输入进行增广，比如说我们训练了 20 个 epoch，那么每个 epoch 里网络看到的输入图像都会略微不同。

# 图像预处理
在 PyTorch 里，我们用 transforms 进行图像预处理。
首先我们定义 training 和 validation 的预处理方式

In [ ]:
from torchvision import transforms
 
# Image transformations
image_transforms = {
    # Train uses data augmentation图像增广
    \\'train\\':
    transforms.Compose([ #compose：将多个transform组合起来使用
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    \\'valid\\':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
#pip install torchvision
#pip install torchvision==0.1.8 #安装制定版本

In [5]:
from torchvision import datasets
from torch.utils.data import DataLoader
 
# Datasets from folders
data = {
    'train\\':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train\\']),
    'valid\\':
    datasets.ImageFolder(root=validdir, transform=image_transforms['valid\\']),
}
 
# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train\\': DataLoader(data['train\\'], batch_size=batch_size, shuffle=True),
    'val\\': DataLoader(data['valid\\'], batch_size=batch_size, shuffle=True)
}

ModuleNotFoundError: No module named 'torchvision'

In [6]:
#我们可以看一下 DataLoader 的输出：
trainiter = iter(dataloaders['train\\'])
features, labels = next(trainiter)
features.shape, labels.shape
(torch.Size([128, 3, 224, 224]), torch.Size([128]))

SyntaxError: unexpected character after line continuation character (<ipython-input-6-acca1da7a139>, line 2)

In [ ]:
mageNet 的预训练模型
PyTorch 自带了很多 ImageNet 上的预训练模型，详细列表见这里。下表是各个模型的性能对比：
本教程将选用 VGG-16 的预训练模型。

In [ ]:
#首先加载预训练模型：
from torchvision import models
model = model.vgg16(pretrained=True)

In [ ]:
#我们只训练这个模型最后的全链接层，所以首先我们要冻结前面的参数：
for param in model.parameters():
    param.requires_grad = False

In [ ]:
#我们的数据集只有 100 个分类，所以要在模型最后面加上几层使得模型的最终输出跟我们的类别数目一样：
import torch.nn as nn
# Add on classifier
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))#？

# 这是什么格式的书写？

In [ ]:
#我们来看一下整个模型：
model.classifier
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4)
    (3): Linear(in_features=256, out_features=100, bias=True)
    (4): LogSoftmax()
  )
)    

In [ ]:
#统计模型参数数量
total_params = sum(p.numel() for p in model.parameters())
print('{total_params:,} 参数总数.\\')

total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print('{total_trainable_params:,} 可训练参数总数.\\')

In [ ]:
#损失函数和优化器
from torch import optim
# Loss and optimizer
criteration = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
#下面是训练的伪代码，大家理解一下其中的思想：
# 伪代码
for epoch in range(n_epochs):
  for data, targets in trainloader:
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()

In [ ]:
#模型训练好后，就可以做预测了：
for data, targets in testloader:
    log_ps = model(data)
    # Convert to probabilities
    ps = torch.exp(log_ps)
ps.shape()
(128, 100)

In [ ]:
#因为我们对所有分类都有输出概率，所以我们要找出概率最大的那个类别来作为最后的预测值
# Find predictions and correct
pred = torch.max(ps, dim=1)
equals = pred == targets
# Calculate accuracy
accuracy = torch.mean(equals)